In [3]:
from entities import *
from constants import TEAMS_DATA, STAT_EFFECTS

import time
import requests
from bs4 import BeautifulSoup, Comment

import dowhy
from tqdm import tqdm, trange

import os
import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings

# Suppress specific pandas FutureWarnings regarding Series.__getitem__
warnings.filterwarnings("ignore", category=FutureWarning, message=".*Series.__getitem__ treating keys as positions is deprecated.*")
warnings.filterwarnings("ignore", category=FutureWarning, message=".*The default of observed=False is deprecated.*")

c:\Users\amgal\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

In [3]:
dfs = []

for team in tqdm(teams.keys()):
    dfs.append(teams[team].causal_df)

df = pd.concat(dfs, ignore_index=True)
df.dropna(inplace=True)
df.head()

100%|██████████| 110/110 [00:00<00:00, 219701.64it/s]


,Team,Opp_Name,Tm,Opp_Score,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,oFG,oFGA,oFG%,o3P,o3PA,o3P%,o2P,o2PA,o2P%,oeFG%,oFT,oFTA,oFT%,oORB,oDRB,oTRB,oAST,oSTL,oBLK,oTOV,oPF
0,Abilene Christian,McMurry,92.0,55.0,34.0,69.0,0.493,6.0,20.0,0.300,28.0,49.0,0.571,0.536,18.0,26.0,0.692,17.0,34.0,51.0,21.0,7.0,7.0,13.0,15.0,18.0,63.0,0.286,8.0,31.0,0.258,10.0,32.0,0.313,0.349,11.0,15.0,0.733,7.0,18.0,25.0,10.0,6.0,5.0,16.0,20.0
1,Abilene Christian,Omaha,73.0,71.0,25.0,57.0,0.439,7.0,21.0,0.333,18.0,36.0,0.500,0.500,16.0,21.0,0.762,9.0,18.0,27.0,15.0,8.0,3.0,14.0,23.0,21.0,45.0,0.467,6.0,19.0,0.316,15.0,26.0,0.577,0.533,23.0,31.0,0.742,3.0,16.0,19.0,10.0,11.0,3.0,15.0,20.0
2,Abilene Christian,SW Adventist,104.0,63.0,43.0,68.0,0.632,9.0,22.0,0.409,34.0,46.0,0.739,0.699,9.0,12.0,0.750,11.0,23.0,34.0,26.0,9.0,3.0,11.0,16.0,22.0,50.0,0.440,4.0,15.0,0.267,18.0,35.0,0.514,0.480,15.0,18.0,0.833,4.0,11.0,15.0,6.0,7.0,0.0,19.0,12.0
3,Abilene Christian,Stephen F. Austin,66.0,76.0,24.0,57.0,0.421,2.0,10.0,0.200,22.0,47.0,0.468,0.439,16.0,19.0,0.842,8.0,27.0,35.0,12.0,4.0,5.0,10.0,21.0,24.0,53.0,0.453,13.0,23.0,0.565,11.0,30.0,0.367,0.575,15.0,26.0,0.577,4.0,22.0,26.0,14.0,5.0,5.0,7.0,15.0
4,Abilene Christian,Texas State,49.0,63.0,17.0,55.0,0.309,5.0,15.0,0.333,12.0,40.0,0.300,0.355,10.0,19.0,0.526,12.0,15.0,27.0,6.0,14.0,5.0,14.0,19.0,21.0,46.0,0.457,5.0,8.0,0.625,16.0,38.0,0.421,0.511,16.0,22.0,0.727,10.0,23.0,33.0,8.0,11.0,9.0,18.0,15.0


## Building our DAG

In [4]:
graph = """
digraph{

Tm;
STL -> oTOV;
STL -> PF -> oFTA -> oFT -> Opp_Score;

BLK -> o2P;
BLK -> o3P;
BLK -> oPF;

TOV -> 3PA -> 3P -> Tm;
TOV -> 2PA -> 2P -> Tm;
TOV -> o3PA -> o3P -> Opp_Score;
TOV -> o2PA -> o2P -> Opp_Score;

ORB -> 3PA;
ORB -> 2PA;

DRB -> o2PA;
DRB -> o3PA;
DRB -> TOV;

Opp_Score;
oSTL -> TOV;
oSTL -> oPF -> FTA -> FT -> Tm;

oBLK -> 2P;
oBLK -> 3P;
oBLK -> PF;

oTOV -> 3PA;
oTOV -> 2PA;
oTOV -> o3PA;
oTOV -> o2PA;

oORB -> o3PA;
oORB -> o2PA;

oDRB -> 2PA;
oDRB -> 3PA;
oDRB -> oTOV;
}
"""

In [5]:
model = dowhy.CausalModel(data=df, graph=graph, treatment="2P",outcome="Tm")

<img src="resources\visualizations\graphviz_graph.png" width="500" />

## Estimating Average Causal Effects

In [6]:
OUTCOMES = ['Tm', 'Opp_Score']

VARIABLES = ['2P',  '2PA',  '3P',  '3PA',  'FT',  'FTA',  'STL',  'BLK',  'TOV',  'PF',  'ORB',  'DRB',  # Team Stats
            'o2P', 'o2PA', 'o3P', 'o3PA', 'oFT', 'oFTA', 'oSTL', 'oBLK', 'oTOV', 'oPF', 'oORB', 'oDRB']  # Opponent Stats

In [ ]:
treatment_effects = {}
treatment_refute = {}

for treatment in tqdm(VARIABLES):
    outcome_effects = {}
    outcome_refute = {}
    for outcome in OUTCOMES:
        
        model = dowhy.CausalModel(
            data=df,
            graph=graph,
            treatment=treatment,
            outcome=outcome
        )

        identified_estimand = model.identify_effect(proceed_when_unidentifiable=False)

        estimate = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.linear_regression",
            target_units="ate"
        )
        
        outcome_effects[outcome] = float(estimate.value)

        if not identified_estimand.estimands:
            outcome_refute[outcome] = 'Skip.'
            continue

        
        common_cause_refute = model.refute_estimate(
            identified_estimand, 
            estimate, 
            method_name="random_common_cause", 
            show_progress_bar=False
        )

        outcome_refute[outcome] = common_cause_refute
        
    treatment_effects[treatment] = outcome_effects
    treatment_refute[treatment] = outcome_refute

    with open(STAT_EFFECTS, 'wb') as f:
        pickle.dump([treatment_effects, treatment_refute], f)

100%|██████████| 24/24 [37:12:40<00:00, 5581.71s/it]    


In [8]:
treatment_effects

{'2P': {'Tm': 2.0156584840313627, 'Opp_Score': 0.0},
 '2PA': {'Tm': 1.1502483831118795, 'Opp_Score': 0.0},
 '3P': {'Tm': 3.0228562864426394, 'Opp_Score': 0.0},
 '3PA': {'Tm': 0.8905147666492041, 'Opp_Score': 0.0},
 'FT': {'Tm': 0.9815989441674375, 'Opp_Score': 0.0},
 'FTA': {'Tm': 0.7573178025133629, 'Opp_Score': 0.0},
 'STL': {'Tm': 0.8589943530837445, 'Opp_Score': -0.753433496527876},
 'BLK': {'Tm': 0.13784012971017034, 'Opp_Score': -0.6418120036151151},
 'TOV': {'Tm': -0.8412514020955797, 'Opp_Score': 0.7406190508702082},
 'PF': {'Tm': 0.0, 'Opp_Score': 1.1022374107567998},
 'ORB': {'Tm': 0.21598461015537396, 'Opp_Score': 0.0},
 'DRB': {'Tm': 0.7581812974921647, 'Opp_Score': -0.8569691436782136},
 'o2P': {'Tm': 0.0, 'Opp_Score': 1.9970590282276248},
 'o2PA': {'Tm': 0.0, 'Opp_Score': 1.0414168089079325},
 'o3P': {'Tm': 0.0, 'Opp_Score': 3.0015172286047687},
 'o3PA': {'Tm': 0.0, 'Opp_Score': 0.9567293811918063},
 'oFT': {'Tm': 0.0, 'Opp_Score': 1.0339679922886873},
 'oFTA': {'Tm': 0.0

# Refuting the Estimates

In [5]:
with open(STAT_EFFECTS, 'rb') as f:
    treatment_effects = pickle.load(f)

In [6]:
treatment_effects[0]

{'2P': {'Tm': 2.0156584840313627, 'Opp_Score': 0.0},
 '2PA': {'Tm': 1.1502483831118795, 'Opp_Score': 0.0},
 '3P': {'Tm': 3.0228562864426394, 'Opp_Score': 0.0},
 '3PA': {'Tm': 0.8905147666492041, 'Opp_Score': 0.0},
 'FT': {'Tm': 0.9815989441674375, 'Opp_Score': 0.0},
 'FTA': {'Tm': 0.7573178025133629, 'Opp_Score': 0.0},
 'STL': {'Tm': 0.8589943530837445, 'Opp_Score': -0.753433496527876},
 'BLK': {'Tm': 0.13784012971017034, 'Opp_Score': -0.6418120036151151},
 'TOV': {'Tm': -0.8412514020955797, 'Opp_Score': 0.7406190508702082},
 'PF': {'Tm': 0.0, 'Opp_Score': 1.1022374107567998},
 'ORB': {'Tm': 0.21598461015537396, 'Opp_Score': 0.0},
 'DRB': {'Tm': 0.7581812974921647, 'Opp_Score': -0.8569691436782136},
 'o2P': {'Tm': 0.0, 'Opp_Score': 1.9970590282276248},
 'o2PA': {'Tm': 0.0, 'Opp_Score': 1.0414168089079325},
 'o3P': {'Tm': 0.0, 'Opp_Score': 3.0015172286047687},
 'o3PA': {'Tm': 0.0, 'Opp_Score': 0.9567293811918063},
 'oFT': {'Tm': 0.0, 'Opp_Score': 1.0339679922886873},
 'oFTA': {'Tm': 0.0

In [7]:
for key, val in list(treatment_effects[1].items()):
    print(f"""
=========================================================
{key}
- Team Score
    {val['Tm']}
- Opponent Score
    {val['Opp_Score']}
""")


2P
- Team Score
    Refute: Add a random common cause
Estimated effect:2.0156584840313627
New effect:2.0156251977720516
p value:0.86

- Opponent Score
    Skipped, due to lack of estimand.


2PA
- Team Score
    Refute: Add a random common cause
Estimated effect:1.1502483831118795
New effect:1.1501868315326191
p value:0.92

- Opponent Score
    Skipped, due to lack of estimand.


3P
- Team Score
    Refute: Add a random common cause
Estimated effect:3.0228562864426394
New effect:3.022915164051245
p value:1.0

- Opponent Score
    Skipped, due to lack of estimand.


3PA
- Team Score
    Refute: Add a random common cause
Estimated effect:0.8905147666492041
New effect:0.8904305638929961
p value:0.84

- Opponent Score
    Skipped, due to lack of estimand.


FT
- Team Score
    Refute: Add a random common cause
Estimated effect:0.9815989441674375
New effect:0.9815775064420253
p value:0.94

- Opponent Score
    Skipped, due to lack of estimand.


FTA
- Team Score
    Refute: Add a random co

# Strength of Schedule Effects

In [29]:
from pathlib import Path
import pyarrow.parquet as pq

team_data = Path.cwd() / 'data' / 'hoopR' / 'team_box_2026.parquet'
player_data = Path.cwd() / 'data' / 'hoopR' / 'player_box_2026.parquet'
basketball_reference_data = Path.cwd() / 'data' / 'Team_Links.csv'

In [30]:
ref_df = pd.read_csv(basketball_reference_data)
ref_df.head()

,Unnamed: 0,School,Link,GameLogs,Pace,ELO
0,0,Abilene Christian,https://www.sports-reference.com/cbb/schools/a...,https://www.sports-reference.com/cbb/schools/a...,68.9,1232.99
1,1,Air Force,https://www.sports-reference.com/cbb/schools/a...,https://www.sports-reference.com/cbb/schools/a...,66.6,1052.74
2,2,Akron,https://www.sports-reference.com/cbb/schools/a...,https://www.sports-reference.com/cbb/schools/a...,72.9,1570.49
3,3,Alabama,https://www.sports-reference.com/cbb/schools/a...,https://www.sports-reference.com/cbb/schools/a...,75.4,1641.07
4,4,Alabama A&M,https://www.sports-reference.com/cbb/schools/a...,https://www.sports-reference.com/cbb/schools/a...,68.8,1159.36


In [31]:
table = pq.read_table(team_data)

teams_df = table.to_pandas()
teams_df.head()

,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,team_short_display_name,team_color,team_alternate_color,team_logo,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,fast_break_points,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,lead_changes,lead_percentage,offensive_rebounds,points_in_paint,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnover_points,turnovers,opponent_team_id,opponent_team_uid,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score
0,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,2539,s:40~l:41~t:2539,san-francisco-dons,San Francisco,Dons,SF,San Francisco Dons,San Francisco,005a36,ffffff,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,away,73,False,15,1,24,9,46.0,26,56,0,14,65.0,15,23,7,6,30,12,36,4,0,0,32.0,6,19,36,0,14,5,14,2541,s:40~l:41~t:2541,santa-clara-broncos,Santa Clara,Broncos,SCU,Santa Clara Broncos,Santa Clara,690b0b,101010,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,88
1,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,2541,s:40~l:41~t:2541,santa-clara-broncos,Santa Clara,Broncos,SCU,Santa Clara Broncos,Santa Clara,690b0b,101010,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,home,88,True,17,0,22,21,51.0,33,65,0,20,77.0,10,13,18,6,61,9,38,12,1,0,46.0,12,26,31,0,7,25,7,2539,s:40~l:41~t:2539,san-francisco-dons,San Francisco,Dons,SF,San Francisco Dons,San Francisco,005a36,ffffff,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,73
2,401825477,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,26,s:40~l:41~t:26,ucla-bruins,UCLA,Bruins,UCLA,UCLA Bruins,UCLA,2774ae,f2a900,https://a.espncdn.com/i/teamlogos/ncaa/500/26.png,away,73,True,10,4,28,10,40.0,23,58,0,13,87.0,20,23,20,1,94,12,30,4,0,0,39.0,7,18,40,0,4,11,4,2483,s:40~l:41~t:2483,oregon-ducks,Oregon,Ducks,ORE,Oregon Ducks,Oregon,00934b,fff41b,https://a.espncdn.com/i/teamlogos/ncaa/500/248...,57
3,401825477,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,2483,s:40~l:41~t:2483,oregon-ducks,Oregon,Ducks,ORE,Oregon Ducks,Oregon,00934b,fff41b,https://a.espncdn.com/i/teamlogos/ncaa/500/248...,home,57,False,9,2,25,7,36.0,21,59,0,17,67.0,6,9,2,1,3,11,24,2,2,0,28.0,9,32,36,0,10,8,10,26,s:40~l:41~t:26,ucla-bruins,UCLA,Bruins,UCLA,UCLA Bruins,UCLA,2774ae,f2a900,https://a.espncdn.com/i/teamlogos/ncaa/500/26.png,73
4,401830830,2026,2,2026-01-28,2026-01-28 22:30:00-05:00,36,s:40~l:41~t:36,colorado-state-rams,Colorado State,Rams,CSU,Colorado State Rams,Colorado St,004c23,c8c372,https://a.espncdn.com/i/teamlogos/ncaa/500/36.png,away,50,False,11,1,13,3,33.0,16,48,0,13,71.0,10,14,6,3,10,11,14,5,3,0,27.0,8,30,24,0,17,16,17,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,a6192e,000000,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,73


In [33]:
df = teams_df
cols = ['team_location', 'team_score', 'opponent_team_location', 'opponent_team_score', 'team_turnovers', 'blocks', 'steals', 'offensive_rebounds', 'defensive_rebounds', 'field_goals_attempted', 'three_point_field_goals_attempted', 'fouls', 'fast_break_points', 'free_throws_attempted']


df = df[df['team_home_away'] == 'away']
df = df[cols]

elo_map = ref_df.set_index('School')['ELO']
df['elo'] = df['team_location'].map(elo_map).fillna(0)
df['opponent_elo'] = df['opponent_team_location'].map(elo_map).fillna(0)

df['elo_delta'] = df['elo'] - df['opponent_elo']
df['score_delta'] = df['team_score'] - df['opponent_team_score']

df = df[(df['elo'] > 0) & (df['opponent_elo'] > 0)]
df

,team_location,team_score,opponent_team_location,opponent_team_score,team_turnovers,blocks,steals,offensive_rebounds,defensive_rebounds,field_goals_attempted,three_point_field_goals_attempted,fouls,fast_break_points,free_throws_attempted,elo,opponent_elo,elo_delta,score_delta
0,San Francisco,73,Santa Clara,88,0,1,4,12,24,56,19,14,9,23,1418.17,1575.11,-156.94,-15
2,UCLA,73,Oregon,57,0,4,4,12,28,58,18,13,10,23,1615.41,1389.56,225.85,16
4,Colorado State,50,San Diego State,73,3,1,5,11,13,48,30,13,3,14,1416.88,1612.65,-195.77,-23
6,Portland,51,Pacific,74,1,1,6,6,20,42,23,14,0,28,1233.42,1339.20,-105.78,-23
12,Houston,79,TCU,70,1,1,6,6,25,54,23,22,6,24,1772.45,1510.74,261.71,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8150,Coppin State,61,Maryland,83,0,2,4,8,21,49,19,17,5,16,958.17,1404.58,-446.41,-22
8152,Morgan State,70,Georgetown,87,1,7,3,11,26,58,18,23,11,24,1120.63,1395.12,-274.49,-17
8160,Northern Arizona,71,Drake,77,0,1,3,15,24,73,24,20,8,12,1113.04,1416.73,-303.69,-6
8164,Omaha,77,Murray State,85,0,2,8,10,25,64,12,21,11,18,1249.39,1444.73,-195.34,-8


In [42]:
graph = """
digraph{

elo_delta;
elo -> elo_delta;
opponent_elo -> elo_delta;

elo_delta -> team_turnovers -> team_score -> score_delta;
elo_delta -> blocks -> team_score -> score_delta;
elo_delta -> steals -> team_score -> score_delta;
elo_delta -> offensive_rebounds -> team_score -> score_delta;
elo_delta -> defensive_rebounds -> team_score -> score_delta;
elo_delta -> field_goals_attempted -> team_score -> score_delta;
elo_delta -> three_point_field_goals_attempted -> team_score -> score_delta;
elo_delta -> fouls -> team_score -> score_delta;
elo_delta -> free_throws_attempted -> team_score -> score_delta;

team_turnovers -> opponent_team_score -> score_delta;
blocks -> opponent_team_score -> score_delta;
steals -> opponent_team_score -> score_delta;
offensive_rebounds -> opponent_team_score -> score_delta;
defensive_rebounds -> opponent_team_score -> score_delta;
field_goals_attempted -> opponent_team_score -> score_delta;
three_point_field_goals_attempted -> opponent_team_score -> score_delta;
fouls -> opponent_team_score -> score_delta;
free_throws_attempted -> opponent_team_score -> score_delta;

}
"""

model = dowhy.CausalModel(data=df, graph=graph, treatment="elo_delta",outcome="score_delta")

<img src="resources\visualizations\graphviz_team_graph.png" width="800" />

In [43]:
OUTCOMES = ['team_turnovers', 'blocks', 'steals', 'offensive_rebounds', 'defensive_rebounds', 'field_goals_attempted', 'three_point_field_goals_attempted', 'fouls', 'free_throws_attempted', 'team_score', 'opponent_team_score', 'score_delta']

ELO_STAT_EFFECTS = Path.cwd() / 'data' / 'team_stat_effects.pkl'

In [44]:
elo_effects = {}
elo_refute = {}
outcome_effects = {}
outcome_refute = {}

for outcome in tqdm(OUTCOMES):
    
    model = dowhy.CausalModel(
        data=df,
        graph=graph,
        treatment='elo_delta',
        outcome=outcome
    )

    identified_estimand = model.identify_effect(proceed_when_unidentifiable=False)

    estimate = model.estimate_effect(
        identified_estimand,
        method_name="backdoor.linear_regression",
        target_units="ate"
    )
    
    outcome_effects[outcome] = float(estimate.value)
    
    common_cause_refute = model.refute_estimate(
        identified_estimand, 
        estimate, 
        method_name="random_common_cause", 
        show_progress_bar=False
    )

    outcome_refute[outcome] = common_cause_refute
    
elo_effects = outcome_effects
elo_refute = outcome_refute

with open(ELO_STAT_EFFECTS, 'wb') as f:
    pickle.dump([elo_effects, elo_refute], f)

100%|██████████| 12/12 [00:14<00:00,  1.17s/it]


In [45]:
elo_effects

{'team_turnovers': -0.0002195586082395229,
 'blocks': 0.0012135203099838954,
 'steals': 0.0014436531373913652,
 'offensive_rebounds': 0.0015873849698326836,
 'defensive_rebounds': 0.007314807712770488,
 'field_goals_attempted': 0.0011141022705771775,
 'three_point_field_goals_attempted': 0.0008158986267687851,
 'fouls': -0.001959951825767803,
 'free_throws_attempted': 0.006257122516846181,
 'team_score': 0.024257842679887176,
 'opponent_team_score': -0.02708223433815249,
 'score_delta': 0.05134007701804988}

In [41]:
for k, v in elo_refute.items():
    print(f"""
=========================================================
{k}
    {v}
""")


team_turnovers
    Refute: Add a random common cause
Estimated effect:-0.0002195586082395229
New effect:-0.00021921845567399067
p value:0.9199999999999999



blocks
    Refute: Add a random common cause
Estimated effect:0.0012135203099838954
New effect:0.0012134554649894413
p value:0.94



steals
    Refute: Add a random common cause
Estimated effect:0.0014436531373913652
New effect:0.0014434223925693511
p value:0.96



offensive_rebounds
    Refute: Add a random common cause
Estimated effect:0.0015873849698326836
New effect:0.0015864956573619971
p value:0.82



defensive_rebounds
    Refute: Add a random common cause
Estimated effect:0.007314807712770488
New effect:0.007314313663481365
p value:0.88



field_goals_attempted
    Refute: Add a random common cause
Estimated effect:0.0011141022705771775
New effect:0.0011159055136955231
p value:0.8999999999999999



three_point_field_goals_attempted
    Refute: Add a random common cause
Estimated effect:0.0008158986267687851
New effect:0.0

In [77]:
OUTCOMES

['team_turnovers',
 'blocks',
 'steals',
 'offensive_rebounds',
 'defensive_rebounds',
 'field_goals_attempted',
 'three_point_field_goals_attempted',
 'fouls',
 'free_throws_attempted',
 'team_score',
 'opponent_team_score',
 'score_delta']

In [81]:
import numpy as np
from scipy.stats import poisson

averages = {}
for o in ['team_turnovers', 'blocks', 'steals', 'offensive_rebounds', 'defensive_rebounds', 'field_goals_attempted', 'fouls',]:
    averages[o] = np.mean(teams_df[o]) / 5


In [82]:
averages

{'team_turnovers': np.float64(0.11942270058708415),
 'blocks': np.float64(0.6755381604696673),
 'steals': np.float64(1.3935665362035226),
 'offensive_rebounds': np.float64(2.190533268101761),
 'defensive_rebounds': np.float64(4.89931506849315),
 'field_goals_attempted': np.float64(11.860445205479452),
 'fouls': np.float64(3.5135763209393347)}